# Error Handling

The code in this notebook helps with handling errors.  Normally, an error in  notebook code causes the execution of the code to stop; while an infinite loop in notebook code causes the notebook to run without end.  This notebook provides two classes to help address these concerns.

**Prerequisites**

* This notebook needs some understanding on advanced concepts in Python, notably 
    * classes
    * the Python `with` statement
    * tracing
    * measuring time
    * exceptions

## Synopsis
<!-- Automatically generated. Do not edit. -->

To [use the code provided in this chapter](Importing.ipynb), write

```python
>>> from debuggingbook.ExpectError import <identifier>
```

and then make use of the following features.


The `ExpectError` class allows you to catch and report exceptions, yet resume execution.  This is useful in notebooks, as they would normally interrupt execution as soon as an exception is raised.  Its typical usage is in conjunction with a `with` clause:

```python
>>> with ExpectError():
>>>     x = 1 / 0
Traceback (most recent call last):
  File "/var/folders/n2/xd9445p97rb3xh7m1dfx8_4h0006ts/T/ipykernel_55153/2664980466.py", line 2, in <cell line: 1>
    x = 1 / 0
ZeroDivisionError: division by zero (expected)

```
The `ExpectTimeout` class allows you to interrupt execution after the specified time.  This is useful for interrupting code that might otherwise run forever.

```python
>>> with ExpectTimeout(5):
>>>     long_running_test()
Start
0 seconds have passed
1 seconds have passed
2 seconds have passed
3 seconds have passed

Traceback (most recent call last):
  File "/var/folders/n2/xd9445p97rb3xh7m1dfx8_4h0006ts/T/ipykernel_55153/1223755941.py", line 2, in <cell line: 1>
    long_running_test()
  File "/var/folders/n2/xd9445p97rb3xh7m1dfx8_4h0006ts/T/ipykernel_55153/3930412460.py", line 4, in long_running_test
    time.sleep(1)
  File "/Users/zeller/Projects/debuggingbook/notebooks/Timeout.ipynb", line 43, in timeout_handler
    raise TimeoutError()
TimeoutError (expected)

```
The exception and the associated traceback are printed as error messages.  If you do not want that, 
use these keyword options:

* `print_traceback` (default True) can be set to `False` to avoid the traceback being printed
* `mute` (default False) can be set to `True` to completely avoid any output.



## Catching Errors

The class `ExpectError` allows to express that some code produces an exception.  A typical usage looks as follows:

```Python
from ExpectError import ExpectError

with ExpectError():
    function_that_is_supposed_to_fail()
```

If an exception occurs, it is printed on standard error; yet, execution continues.

In [1]:
import bookutils.setup

In [2]:
import traceback
import sys

In [3]:
from types import FrameType, TracebackType

In [4]:
# ignore
from typing import Union, Optional, Callable, Any

In [12]:
from ExpectError import ExpectError, ExpectTimeout

Here's an example:

In [5]:
def fail_test() -> None:
    # Trigger an exception
    x = 1 / 0

In [6]:
with ExpectError():
    fail_test()

Traceback (most recent call last):
  File "C:\Users\siyuey\AppData\Local\Temp\ipykernel_36432\1235320646.py", line 2, in <module>
    fail_test()
  File "C:\Users\siyuey\AppData\Local\Temp\ipykernel_36432\278441162.py", line 3, in fail_test
    x = 1 / 0
ZeroDivisionError: division by zero (expected)


In [7]:
with ExpectError(print_traceback=False):
    fail_test()

ZeroDivisionError: division by zero (expected)


We can specify the type of the expected exception. This way, if something else happens, we will get notified.

In [8]:
with ExpectError(ZeroDivisionError):
    fail_test()

Traceback (most recent call last):
  File "C:\Users\siyuey\AppData\Local\Temp\ipykernel_36432\1259188418.py", line 2, in <module>
    fail_test()
  File "C:\Users\siyuey\AppData\Local\Temp\ipykernel_36432\278441162.py", line 3, in fail_test
    x = 1 / 0
ZeroDivisionError: division by zero (expected)


In [9]:
with ExpectError():
    with ExpectError(ZeroDivisionError):
        some_nonexisting_function()  # type: ignore

Traceback (most recent call last):
  File "C:\Users\siyuey\AppData\Local\Temp\ipykernel_36432\2242794116.py", line 3, in <module>
    some_nonexisting_function()  # type: ignore
  File "C:\Users\siyuey\AppData\Local\Temp\ipykernel_36432\2242794116.py", line 3, in <module>
    some_nonexisting_function()  # type: ignore
NameError: name 'some_nonexisting_function' is not defined (expected)


## Catching Timeouts

The class `ExpectTimeout(seconds)` allows expressing that some code may run for a long or infinite time; execution is thus interrupted after `seconds` seconds.  A typical usage looks as follows:

```Python
from ExpectError import ExpectTimeout

with ExpectTimeout(2) as t:
    function_that_is_supposed_to_hang()
```

If an exception occurs, it is printed on standard error (as with `ExpectError`); yet, execution continues.

Should there be a need to cancel the timeout within the `with` block, `t.cancel()` will do the trick.

The implementation uses `sys.settrace()`, as this seems to be the most portable way to implement timeouts.  It is not very efficient, though.  Also, it only works on individual lines of Python code and will not interrupt a long-running system function.

In [10]:
import sys
import time

Here's an example:

In [13]:
def long_running_test() -> None:
    print("Start")
    for i in range(10):
        time.sleep(1)
        print(i, "seconds have passed")
    print("End")

In [14]:
with ExpectTimeout(5, print_traceback=False):
    long_running_test()

Start
0 seconds have passed
1 seconds have passed
2 seconds have passed
3 seconds have passed


TimeoutError (expected)


Note that it is possible to nest multiple timeouts.

In [15]:
with ExpectTimeout(5, print_traceback=False):
    with ExpectTimeout(3, print_traceback=False):
        long_running_test()
    long_running_test()

Start
0 seconds have passed
1 seconds have passed


TimeoutError (expected)


Start
0 seconds have passed


TimeoutError (expected)


That's it, folks – enjoy!

## Synopsis

The `ExpectError` class allows you to catch and report exceptions, yet resume execution.  This is useful in notebooks, as they would normally interrupt execution as soon as an exception is raised.  Its typical usage is in conjunction with a `with` clause:

In [17]:
with ExpectError():
    x = 1 / 0

Traceback (most recent call last):
  File "/var/folders/n2/xd9445p97rb3xh7m1dfx8_4h0006ts/T/ipykernel_55153/2664980466.py", line 2, in <cell line: 1>
    x = 1 / 0
ZeroDivisionError: division by zero (expected)


The `ExpectTimeout` class allows you to interrupt execution after the specified time.  This is useful for interrupting code that might otherwise run forever.

In [18]:
with ExpectTimeout(5):
    long_running_test()

Start
0 seconds have passed
1 seconds have passed
2 seconds have passed
3 seconds have passed


Traceback (most recent call last):
  File "/var/folders/n2/xd9445p97rb3xh7m1dfx8_4h0006ts/T/ipykernel_55153/1223755941.py", line 2, in <cell line: 1>
    long_running_test()
  File "/var/folders/n2/xd9445p97rb3xh7m1dfx8_4h0006ts/T/ipykernel_55153/3930412460.py", line 4, in long_running_test
    time.sleep(1)
  File "/Users/zeller/Projects/debuggingbook/notebooks/Timeout.ipynb", line 43, in timeout_handler
    raise TimeoutError()
TimeoutError (expected)


The exception and the associated traceback are printed as error messages.  If you do not want that, 
use these keyword options:

* `print_traceback` (default True) can be set to `False` to avoid the traceback being printed
* `mute` (default False) can be set to `True` to completely avoid any output.

## Lessons Learned

* With the `ExpectError` class, it is very easy to handle errors without interrupting notebook execution.